In [2]:
from pyomo.environ import *
import itertools
import pandas as pd

### Problem

$$\text{Maximize } z = 20x_1 + 40x_2 + 20x_3 + 15x_4 + 30x_5$$

#### Restrictions

Year 1:  $ 5x_1 + 4x_2 + 3x_3 + 7x_4 + 8x_5 \leq 25$

Year 2:  $ 1x_1 + 7x_2 + 9x_3 + 4x_4 + 6x_5 \leq 25$

Year 3:  $ 8x_1 + 10x_2 + 2x_3 + 1x_4 + 10x_5 \leq 25$

In [ ]:
def analyze_combinations():
    
    combinations = list(itertools.product([0,1], repeat=5))
    
    results = []
    for comb in combinations:
        x1, x2, x3, x4, x5 = comb
        total_return = 20*x1 + 40*x2 + 20*x3 + 15*x4 + 30*x5
        
        year1 = 5*x1 + 4*x2 + 3*x3 + 7*x4 + 8*x5
        year2 = 1*x1 + 7*x2 + 9*x3 + 4*x4 + 6*x5
        year3 = 8*x1 + 10*x2 + 2*x3 + 1*x4 + 10*x5
        
        # constraint violations
        violates_year1 = year1 > 25
        violates_year2 = year2 > 25
        violates_year3 = year3 > 25
        any_violation = violates_year1 or violates_year2 or violates_year3
        is_feasible = not any_violation
        

        selected_projects = [f'Project {i+1}' for i, val in enumerate(comb) if val == 1]
        
        results.append({
            'Combination': f"{x1}{x2}{x3}{x4}{x5}",
            'Selected Projects': ', '.join(selected_projects) if selected_projects else 'None',
            'Total Return ($M)': total_return,
            'Year 1 ($M)': year1,
            'Year 2 ($M)': year2, 
            'Year 3 ($M)': year3,
            'Violates Year 1': 'Yes' if violates_year1 else 'No',
            'Violates Year 2': 'Yes' if violates_year2 else 'No',
            'Violates Year 3': 'Yes' if violates_year3 else 'No',
            'Feasible': 'Yes' if is_feasible else 'No'
        })
    
    df = pd.DataFrame(results)
    
    print("\nCombinations that VIOLATE constraints:")
    print(df[df['Feasible'] == 'No'][['Combination', 'Selected Projects', 'Total Return ($M)', 
                                     'Year 1 ($M)', 'Year 2 ($M)', 'Year 3 ($M)']])
    
    print("\n" + "="*50)
    print("\nFEASIBLE combinations (sorted by highest return):")
    feasible = df[df['Feasible'] == 'Yes'].sort_values('Total Return ($M)', ascending=False)
    print(feasible[['Combination', 'Selected Projects', 'Total Return ($M)', 
                   'Year 1 ($M)', 'Year 2 ($M)', 'Year 3 ($M)']])
    
    return df

In [4]:
project_selection_model = ConcreteModel()

#Problem data definition:
projects = [1, 2, 3, 4, 5]
returns = {1: 20, 2: 40, 3: 20, 4: 15, 5: 30}
cost = {
    1: [5, 1, 8],
    2: [4, 7, 10],
    3: [3, 9, 2],
    4: [7, 4, 1],
    5: [8, 6, 10]
}

annual_budget = [25, 25, 25]

#Decision variables
project_selection_model.x = Var(projects, within=Binary)

#OBJECTIVE FUNCTION: maximize total return
project_selection_model.obj = Objective(
    expr=sum(returns[j] * project_selection_model.x[j] for j in projects),
    sense=maximize
)

In [5]:
#Applying the restrictions
project_selection_model.ano1_constraint = Constraint(
    expr=sum(cost[j][0] * project_selection_model.x[j] for j in projects) <= annual_budget[0]
)
project_selection_model.ano2_constraint = Constraint(
    expr=sum(cost[j][1] * project_selection_model.x[j] for j in projects) <= annual_budget[1]
)
project_selection_model.y3_constraint = Constraint(
    expr=sum(cost[j][2] * project_selection_model.x[j] for j in projects) <= annual_budget[2]
)

In [6]:
solver = SolverFactory('glpk')
result = solver.solve(project_selection_model)

print("Optimization Status:", result.solver.status)
print("Solution Status:", result.solver.termination_condition)

print("\nSelected Projects:")
for j in projects:
    if project_selection_model.x[j].value == 1:
        print(f"Project {j}: Selected")

print("\nTotal Return:", project_selection_model.obj())

Optimization Status: ok
Solution Status: optimal

Selected Projects:
Project 1: Selected
Project 2: Selected
Project 3: Selected
Project 4: Selected

Total Return: 95.0


In [ ]:
results_df = analyze_combinations()


Combinations that VIOLATE constraints:
   Combination                                  Selected Projects  \
15       01111         Project 2, Project 3, Project 4, Project 5   
25       11001                    Project 1, Project 2, Project 5   
27       11011         Project 1, Project 2, Project 4, Project 5   
29       11101         Project 1, Project 2, Project 3, Project 5   
31       11111  Project 1, Project 2, Project 3, Project 4, Pr...   

    Total Return ($M)  Year 1 ($M)  Year 2 ($M)  Year 3 ($M)  
15                105           22           26           23  
25                 90           17           14           28  
27                105           24           18           29  
29                110           20           23           30  
31                125           27           27           31  


FEASIBLE combinations (sorted by highest return):
   Combination                           Selected Projects  Total Return ($M)  \
30       11110  Project 1, Project